In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
df_ha = pd.read_csv("../data/headache_record2.csv")

In [ ]:
df_ha.head()

In [ ]:
df_ha["label"] = 1
df_ha["日時"] = df_ha["日時"].map(lambda x: x.split()[0])

In [ ]:
df_ha.tail()

In [ ]:
pos_list = []
for i in range(len(df_ha)):
    pos_list.append(df_ha["日時"][i])

In [ ]:
for day in range(11, 32):
    date = "2019/12/{}".format(day)
    if not date in pos_list:
        df_ha = df_ha.append(pd.Series([date, "", "", 0], index=df_ha.columns), ignore_index=True)
        print(date)

In [ ]:
df_ha

In [ ]:
month = range(1, 13)
for m in month:
    if m in [2, 4, 6, 9, 11]:
        date_gene = range(1, 31)
    else:
        date_gene = range(1, 32)
    for d in date_gene:
        date = "2020/{0}/{1}".format(m, d)
        if not date in pos_list:
            df_ha = df_ha.append(pd.Series([date, "", "", 0], index=df_ha.columns), ignore_index=True)

In [ ]:
month = [1]
for m in month:
    if m in [2, 4, 6, 9, 11]:
        date_gene = range(1, 12)
    else:
        date_gene = range(1, 12)
    for d in date_gene:
        date = "2021/{0}/{1}".format(m, d)
        if not date in pos_list:
            df_ha = df_ha.append(pd.Series([date, "", "", 0], index=df_ha.columns), ignore_index=True)

In [ ]:
df_ha = df_ha.sort_values("日時")

In [ ]:
df_w = pd.read_csv("./../data/data2.csv", encoding="SHIFT-JIS")

In [ ]:

df_w.head()

In [ ]:
df_w["日時"] = df_w["日時"].map(lambda x: x.replace("年", "/").replace("月", "/").replace("日", ""))

In [ ]:
df = pd.merge(df_ha, df_w)

In [ ]:
df_p = pd.read_csv("./../data/period2.csv")
df_p.head()

In [ ]:
import datetime

In [ ]:
y, m, d = map(int, df_p["startdate"][0].split("/"))
datetime.date(y, m, d)

In [ ]:
def get_passed_st(date):
    y, m, d = map(int, date.split("/"))
    d_o = datetime.date(y, m, d)
    tmp_ans = 100
    for i in range(len(df_p)):
        y, m, d = map(int, df_p["startdate"][i].split("/"))
        dc = datetime.date(y, m, d)
        day_delta = (d_o - dc).days
        if day_delta <= 0:
            return tmp_ans
        if tmp_ans > day_delta:
            tmp_ans = day_delta
    return tmp_ans

def get_passed_en(date):
    y, m, d = map(int, date.split("/"))
    d_o = datetime.date(y, m, d)
    tmp_ans = 100
    for i in range(len(df_p)):
        y, m, d = map(int, df_p["enddate"][i].split("/"))
        dc = datetime.date(y, m, d)
        day_delta = (d_o - dc).days
        if day_delta <= 0:
            return tmp_ans
        if tmp_ans > day_delta:
            tmp_ans = day_delta
    return tmp_ans

In [ ]:
get_passed_en("2019/12/10")

In [ ]:
df.head()

In [ ]:
df["気温差"] = df["最高気温"] - df["最低気温"]

In [ ]:
df.head()

In [ ]:
df["day_passed"] = df["日時"].map(get_passed_st)
df["day_passed_en"] = df["日時"].map(get_passed_en)

In [ ]:
df.tail()

In [ ]:
df["1d_ha"] = 0
for i in range(len(df)-1):
    if df["label"][i] == 1:
        df["1d_ha"][i+1] = 1
df.head()

In [ ]:
df["3d_ha"] = 0
for i in range(len(df)):
    if i == 0:
        continue
    if i == 1:
        df["3d_ha"][i] = df["label"][0]
    elif i == 2:
        df["3d_ha"][i] = df["label"][0] + df["label"][1]
    else:
        df["3d_ha"][i] = df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
df.head()

In [ ]:
df["7d_ha"] = 0
for i in range(len(df)):
    if i == 0:
        continue
    if i == 1:
        df["7d_ha"][i] = df["label"][0]
    elif i == 2:
        df["7d_ha"][i] = df["label"][0] + df["label"][1]
    elif i == 3:
        df["7d_ha"][i] = df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
    elif i == 4:
        df["7d_ha"][i] = df["label"][i-4] + df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
    elif i == 5:
        df["7d_ha"][i] = df["label"][i-5] + df["label"][i-4] + df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
    elif i == 6:
        df["7d_ha"][i] = df["label"][i-6] + df["label"][i-5] + df["label"][i-4] + df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
    else:
        df["7d_ha"][i] = df["label"][i-7] + df["label"][i-6] + df["label"][i-5] + df["label"][i-4] + df["label"][i-3] + df["label"][i-2] + df["label"][i-1]
df.head()

In [ ]:
df["delta_pressure"] = 0
for i in range(len(df)):
    if i == 0:
        continue
    else:
        df["delta_pressure"][i] = df["平均現地気圧"][i] - df["平均現地気圧"][i-1]
df.head()

In [ ]:
df["天気概況昼"].value_counts()

In [ ]:
df["天気概況夜"].value_counts()

In [ ]:
y = np.array(df["label"])

In [ ]:
X = df[["最高気温", "気温差", "day_passed", "平均現地気圧", "1d_ha", "3d_ha", "7d_ha", "delta_pressure"]]

In [ ]:
X = df[["day_passed", "day_passed_en"]]

In [ ]:
X = df[["平均現地気圧"]]

In [ ]:

sns.pairplot(data=df, hue='label')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
lgbm_params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'auc',
    # Fatal の場合出力
    'verbosity': -1,
}

In [ ]:
X_train

In [ ]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=100 # 最大イテレーション回数指定
                 )

In [ ]:
model.predict(X_test)

In [ ]:
# maketest
df = pd.read_csv("./../data/test.csv")
X_test["最高気温"] = X_test["最高気温"].map(float)
X_test["最低気温"] = X_test["最低気温"].map(float)
X_test["降水量"] = X_test["降水量"].map(float)
X_test["気温差"] = X_test["気温差"].map(float)
X_test

In [ ]:
df_predict = pd.read_csv("./../data/test.csv")
df_predict

In [ ]:

model.predict(df_predict[["最高気温", "気温差", "day_passed", "平均現地気圧", "1d_ha", "3d_ha", "7d_ha", "delta_pressure"]])

# todo
天気のonehot-encoding  
交差検証  
データ水増し、アンダーサンプリング  
パラメタチューニング  
過去に1日/3日/7日以内に頭痛になったかどうか  
気圧の変化  
その他のデルタ特徴量

# done
平均気圧の変化